# TAREA I

## 1. Estudio De Ingeniería
### I. Límites Admisibles de Operación
### II. Tipos de Modelados De Líneas

## 2. MODELO A

En base a la información suministrada para el **Sistema A** y mostrada en la figura se plantea el esquema con Panda Power. Y se realiza el flujo de carga respectivo para la carga que cambia en el rango $\pm 50\%$ de la potencia indicada.

<div style="text-align: center;">
  <img src="IMAGENES/MODELO_A.PNG" alt="Descripción de la imagen" width="200"/>
</div>

**OBSERVACIÓN:** Puesto que la carga posee un rango de operación, se calcula previamente este rango de la siguiente manera:

  $$Rango\ Carga = [1200 \times 0.5\ \text{MVA},\ 1200 \times 1.5\ \text{MVA}] = [600\ \text{MVA},\ 1800\ \text{MVA}]$$

In [69]:
import pandapower as pp
import math as mm

#Variables Globales
Tensiones = []
Paso = 100 #MVA
#max_iteraciones = 100 
#Parámetros de Línea
r1 = 0.02  #(Ohm/km)
x1 = 0.115 #(Ohm/km)
B1 = 19.1  #(nF/km)
Imax1 = 1  #(kA)

for i in range(600,1800,100): 
    net1 = pp.create_empty_network()
    #Barras
    b1_1 = pp.create_bus(net1, vn_kv=220, name="Barra 1")
    b2_1 = pp.create_bus(net1, vn_kv=220, name="Barra 2")
    #Calculo De Potencias En Carga
    FP1 = 0.9
    theta1 = mm.acos(FP1)
    P1 = round(i*mm.cos(theta1),1)
    Q1 = round(i*mm.sin(theta1),1)
    #Elementos De Barra
    pp.create_ext_grid(net1, bus=b1_1, vm_pu=1.00, name="Fuente Externa") #FUENTE
    pp.create_load(net1, bus=b2_1, p_mw=P1, q_mvar=Q1, name="Carga 1")
    #LINEAS
    pp.create_line_from_parameters(net1, name='Linea 1-2(I)', length_km=500, from_bus=b1_1, to_bus=b2_1,
                                   r_ohm_per_km=r1, x_ohm_per_km=x1, c_nf_per_km=B1, max_i_ka=Imax1)
    pp.create_line_from_parameters(net1, name='Linea 1-2(II)', length_km=500, from_bus=b1_1, to_bus=b2_1,
                                   r_ohm_per_km=r1, x_ohm_per_km=x1, c_nf_per_km=B1, max_i_ka=Imax1)
    #Flujo De Energía
    pp.runpp(net1)
    #tension_barra2 = net1.res_bus.at[1,'vm_pu']
    #Tensiones.append(tension_barra2)

#print(Tensiones)  

LoadflowNotConverged: Power Flow nr did not converge after 10 iterations!

## 3. MODELO B

In [ ]:
#Algo asi hay que hacer en la tarea
import cmath
import numpy as np
#Primero que nada necesitamos encontrar el valor de las matrices locales de las lineas, por lo que aquí se colocan los datos de la linea y despues se modelan por el modelo Pi para luego poder sumarlas al Y_buss
Ybuss = np.zeros((6, 6), dtype=complex)
Lista_datos_Lineas= [("1-2",0.031,0.126,0.062),
                     ("1-5",0.031,0.126,0.062),
                     ("2-3",0.063,0.252,0.122),
                     ("3-5",0.053,0.210,0.102),
                     ("3-6",0.084,0.236,0.164)]; # (linea,r,x,b)
for i in Lista_datos_Lineas:
    L=150
    Linea=i[0].split("-")
    Linea_a=int(Linea[0])-1
    Linea_b=int(Linea[1])-1
    z = complex(i[1],i[2])
    y = complex(0,i[3])
    Zc = cmath.sqrt(z/y)
    gama = cmath.sqrt(z*y)
    Z_modelo=Zc*gama*L
    Y_Serie=1/Z_modelo

    Y_Shunt=gama*L/(Zc*2)
    Y_Shunt=Y_Shunt.imag*1j
    
    A=Y_Serie+Y_Shunt
    B=-Y_Serie
    C=B
    D=A

    coordenadas = [(Linea_a, Linea_a), (Linea_a, Linea_b), 
                   (Linea_b, Linea_a), (Linea_b, Linea_b)]
    valores_a_sumar = [A, B, C, D]
    for coord, valor in zip(coordenadas, valores_a_sumar):
        fila, columna = coord
        Ybuss[fila, columna] += valor

############################Trafos######################################

Lista_datos_Trafos= [("4-5",0,0.1,0),
                     ("4-6",0,0.1,0)]; # (linea,r,x,b)
for i in Lista_datos_Trafos:
    Linea=i[0].split("-")
    Linea_a=int(Linea[0])-1
    Linea_b=int(Linea[1])-1

    Z_trafo=complex(i[1],i[2])
    Y_trafo=1/Z_trafo
    A=Y_trafo
    B=-A
    C=B
    D=A

    valores_a_sumar = [A, B, C, D]
    coordenadas = [(Linea_a, Linea_a), (Linea_a, Linea_b), (Linea_b, Linea_a), (Linea_b, Linea_b)]
    for coord, valor in zip(coordenadas, valores_a_sumar):
        fila, columna = coord
        Ybuss[fila, columna] += valor

############################Generación######################################

Lista_datos_Trafos= [("1",0,1.1,0),
                     ("3",0,1.3,0)]; # (linea,r,x,b)
for i in Lista_datos_Trafos:
    Linea=i[0].split("-")
    Linea_a=int(Linea[0])-1
    Z_trafo=complex(i[1],i[2])
    Y_trafo=1/Z_trafo

    A=Y_trafo
    coordenadas = [(Linea_a, Linea_a)]
    valores_a_sumar = [A]
    for coord, valor in zip(coordenadas, valores_a_sumar):
        fila, columna = coord
        Ybuss[fila, columna] += valor
print(Ybuss)


In [ ]:
for p in range(1,10,1):
    print(p)